## Package Imports

In [1]:
import sys
from pathlib import Path
import os
import numpy as np

In [2]:
project_working_dir = str(Path(sys.path[0]).parent)
sys.path += [project_working_dir]
os.chdir(project_working_dir)

## Local Imports

In [3]:
from src.utils.read_obj import parse_obj
from src.utils.file_io import read_json
from src.simulation.setup.extract_clothing_vertex_data import extract_all_piece_vertices
from src.parameters import AVATAR_SCALING

In [4]:
avatar_mesh = parse_obj('./assets/BodyMesh.obj', './assets/BodyAnnotations.json')
avatar_mesh.scale_vertices(AVATAR_SCALING)

clothing_data = read_json('./assets/sewing_shirt.json')
all_pieces, sewing = extract_all_piece_vertices(clothing_data, avatar_mesh)

## Profile cpu version

In [5]:
trimesh = avatar_mesh.trimesh

In [6]:
vertices = all_pieces["L-1"].mesh.vertices_3d

In [7]:
%timeit trimesh.contains(vertices)

2.27 s ± 452 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit trimesh.nearest.on_surface(vertices)

509 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
trimesh.is_watertight

True

In [10]:
trimesh.fix_normals()

In [14]:
np.degrees(trimesh.face_adjacency_angles).min()

0.0008763332504599916

In [15]:
np.degrees(trimesh.face_adjacency_angles).max()

177.68969464371781

## Save numpy vertices

In [13]:
def save_numpy_array(filename: str, arr: np.ndarray):
    np.save(f'{project_working_dir}/profiling/numpy/{filename}', arr)

In [14]:
vertices.shape

(3205, 3)

In [15]:
save_numpy_array('piece_vertices.npy', vertices)

In [16]:
len(set(trimesh.faces.flatten())) / len(trimesh.faces.flatten())

0.16700542005420055

In [15]:
len(trimesh.faces.flatten())

5904

In [18]:
triangle_vertices = np.array([
    [trimesh.vertices[i], trimesh.vertices[j], trimesh.vertices[k]] for i, j, k in trimesh.faces
], dtype=np.float32)

In [19]:
triangle_vertices.shape

(1968, 3, 3)

In [20]:
save_numpy_array('triangle_vertices.npy', triangle_vertices)

In [21]:
save_numpy_array('triangle_normals.npy', trimesh.face_normals)

In [22]:
trimesh.face_normals.shape

(1968, 3)

In [23]:
in_shape = trimesh.contains(vertices)

In [24]:
in_shape.sum()

50

In [25]:
np.where(in_shape)[0]

array([1680, 1683, 1741, 1798, 2245, 2298, 2299, 2300, 2301, 2302, 2303,
       2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355,
       2356, 2357, 2358, 2359, 2360, 2361, 2362, 2406, 2407, 2408, 2456,
       2457, 2458, 2459, 2498, 2499, 2501, 2549, 2600, 2609, 2651, 2655,
       2690, 2701, 2741, 2742, 2793, 2945], dtype=int64)